In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import matplotlib.ticker as mtick
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
from shapely.geometry import Polygon

import geopandas as gpd
from shapely.geometry import Polygon
from itertools import product
import warnings
warnings.filterwarnings("ignore")

import shapely

In [ ]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [ ]:
df.head()

In [ ]:
#fill na values with 0

df_geographic_unique = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].fillna(0)
df_geographic_unique['countyCode'] = df_geographic_unique['countyCode'].fillna(0)
df_geographic_unique['censusTract'] = df_geographic_unique['censusTract'].fillna(0)
df_geographic_unique['censusBlockGroupFips'] = df_geographic_unique['censusBlockGroupFips'].fillna(0)
df_geographic_unique['latitude'] = df_geographic_unique['latitude'].fillna(0)
df_geographic_unique['longitude'] = df_geographic_unique['longitude'].fillna(0)

df_geographic_unique = df_geographic_unique.drop_duplicates()

In [ ]:
#convert the units to string to use mapping and fix the lenght of each units

df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].astype(int).astype(str)
df_geographic_unique['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_geographic_unique['reportedZipCode']]

df_geographic_unique['censusBlockGroupFips'] = [str(int(float(i))) for i in df_geographic_unique['censusBlockGroupFips']]
df_geographic_unique['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_geographic_unique['censusBlockGroupFips']]

df_geographic_unique['countyCode'] = [str(int(float(i))) for i in df_geographic_unique['countyCode']]
df_geographic_unique['countyCode'] = [censusBG.zfill(5) for censusBG in df_geographic_unique['countyCode']]

df_geographic_unique['censusTract'] = [str(int(float(i))) for i in df_geographic_unique['censusTract']]
df_geographic_unique['censusTract'] = [censusBG.zfill(11) for censusBG in df_geographic_unique['censusTract']]

In [ ]:
# Define bins and labels for yearOfLoss_1990_2021

bins_1980_2021 = [df['yearOfLoss'].min(), 1980, 1990, 2000, 2010, 2020, df['yearOfLoss'].max() + 1]
labels_1980_2021 = [0, 1980, 1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)


df_geographic_unique = df_geographic_unique.drop_duplicates()

In [ ]:


# Adjusting the bins so that 2011 falls into the 2012 bin
custom_bins = [0, 2000] + list(range(2010, 2023)) + [2024]

# Step 2: Define the labels.
# Adjusting the labels to reflect the new bin structure
custom_labels = [0, 2000] + list(range(2010, 2023))  # The label 2022 will apply to both 2022 and 2023.

# Ensure the number of labels is one less than the number of bin edges.
assert len(custom_bins) == len(custom_labels) + 1, "Number of bin labels must be one fewer than the number of bin edges."

# Step 3: Apply the custom binning and create the 'zip_year_bin' column.
df_geographic_unique['zip_year_bin'] = pd.cut(
    df_geographic_unique['yearOfLoss'],
    bins=custom_bins,
    labels=custom_labels,
    right=False,  # Ensuring the rightmost edge is exclusive.
    include_lowest=True,  # Including the leftmost edge.
).astype(int)  # Converting the resulting categories to integers for convenience.

## Read shapefiles

In [ ]:
#State shapefile

states = pygris.states()

state_df = states[['STUSPS', 'NAME', 'geometry']]

In [ ]:
#Lat-Long shapefile
df_read = pd.read_parquet("C:/Users/Jorda/Box/Flood Damage PredictionProject/Dataset/lat_long_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
lat_long_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [ ]:
# Blockgroup shapefile

chunk_size = 40000 
chunks = [x for x in range(0, 120000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Jorda/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
BG_df= pd.concat(gdf_list, ignore_index=True)

In [ ]:
chunk_size = 50000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 300000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Jorda/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)
    
# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
zipcode_df = pd.concat(gdf_list, ignore_index=True)

In [ ]:
# Read shapefile of county code
df_read = pd.read_parquet("C:/Users/Jorda/Box/Flood Damage PredictionProject/Dataset/County_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
County_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [ ]:
# Read shapefile of census tract code
chunk_size = 30000 
chunks = [x for x in range(0, 60000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Jorda/Box/Flood Damage PredictionProject/Dataset/Tract_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
Tract_df= pd.concat(gdf_list, ignore_index=True)

### Mapping Geometries to the Dataset

In [ ]:
state_df.rename(columns={'geometry': 'geometry_state'}, inplace=True)
lat_long_df.rename(columns={'geometry': 'geometry_lat_long'}, inplace=True)
BG_df.rename(columns={'geometry': 'geometry_BG'}, inplace=True)
zipcode_df.rename(columns={'geometry': 'geometry_zipcode'}, inplace=True)
County_df.rename(columns={'geometry': 'geometry_county'}, inplace=True)
Tract_df.rename(columns={'geometry': 'geometry_tract'}, inplace=True)

In [ ]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_lat_long'] = df_geographic_unique.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

lat_long_df.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
BG_df.set_index(['GEOID'], inplace=True)
df_geographic_unique['geometry_BG'] = df_geographic_unique.set_index(['censusBlockGroupFips']).index.map(BG_df['geometry_BG'])

BG_df.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
zipcode_df.set_index(['ZIPcode', 'year'], inplace=True)
df_geographic_unique['geometry_zipcode'] = df_geographic_unique.set_index(['reportedZipCode', 'zip_year_bin']).index.map(zipcode_df['geometry_zipcode'])

zipcode_df.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
state_df.set_index(['STUSPS'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_state'] = df_geographic_unique.set_index(['state']).index.map(state_df['geometry_state'])

state_df.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
County_df.set_index(['CountyID'], inplace=True)
df_geographic_unique['geometry_county'] = df_geographic_unique.set_index(['countyCode']).index.map(County_df['geometry_county'])

# Resetting the index of County_df (return to multi-index)
County_df.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
Tract_df.set_index(['censusTractID'], inplace=True)
df_geographic_unique['geometry_tract'] = df_geographic_unique.set_index(['censusTract']).index.map(Tract_df['geometry_tract'])

# Resetting the index of County_df (return to multi-index)
Tract_df.reset_index(inplace=True)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Assuming you have a GeoDataFrame named df_geographic_unique
# Make a copy of the original GeoDataFrame to avoid modifying it
df_copy = df_geographic_unique.copy()

# Extract the original geometry of the first observation at index 1
original_geometry = df_copy['geometry_zipcode'].iloc[1]

# Calculate the original area
original_area = original_geometry.area

# Define a target area which is 10% greater than the original area
target_area = original_area * 1.10

# Initialize the buffer distance and the buffered area
buffer_distance = 0
buffered_area = 0

# Incrementally increase the buffer distance until the area of the buffered shape
# is at least as large as the target area
while buffered_area < target_area:
    buffer_distance += 0.00001  # Increment buffer_distance by a small amount
    buffered_geometry = original_geometry.buffer(buffer_distance)
    buffered_area = buffered_geometry.area

# Create a GeoDataFrame with just the buffered geometry
gdf_buffered = gpd.GeoDataFrame({'geometry': [buffered_geometry]}, crs="EPSG:4326")

# Create a GeoDataFrame with just the original geometry
gdf_original = gpd.GeoDataFrame({'geometry': [original_geometry]}, crs="EPSG:4326")

# Create subplots for before, after, and combined plots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))  # Adjusted for 3 subplots

# Plot the original geometry at index 1 before the buffer using GeoPandas
gdf_original.plot(ax=axes[0], color='blue', alpha=0.5)
axes[0].set_title('Original Geometry (EPSG:4326)')

# Plot only the buffered geometry
gdf_buffered.plot(ax=axes[1], color='red', alpha=0.5)
axes[1].set_title('Buffered Geometry (EPSG:4326)')

# Plot the buffered geometry first, then the original geometry on the same subplot for comparison
gdf_buffered.plot(ax=axes[2], color='red', alpha=0.3)  # Use a lighter alpha for the buffer
gdf_original.plot(ax=axes[2], color='blue', alpha=0.5)
axes[2].set_title('Comparison of Original and Buffered Geometries (EPSG:4326)')

# Highlight the buffer by plotting the difference between the buffered and original geometries
buffer_difference = gdf_buffered['geometry'].difference(gdf_original['geometry'].iloc[0])
gpd.GeoDataFrame({'geometry': buffer_difference}, crs="EPSG:4326").plot(ax=axes[2], color='yellow', alpha=0.5)

# Show the plots
plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()

# Print the areas before and after the buffer
print(f"Original Area: {original_area}")
print(f"Buffered Area: {buffered_area}")


In [ ]:
zipcode_df.columns

### Adding the inconsistent flags to the dataset

In [ ]:
df_1_inconsistent = pd.read_parquet('C:/Users/Jorda/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_1.parquet.gzip')
df_2_inconsistent = pd.read_parquet('C:/Users/Jorda/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_2.parquet.gzip')
df_3_inconsistent = pd.read_parquet('C:/Users/Jorda/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_3.parquet.gzip')
df_4_inconsistent = pd.read_parquet('C:/Users/Jorda/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_4.parquet.gzip')
df_5_inconsistent = pd.read_parquet('C:/Users/Jorda/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_5.parquet.gzip')
df_6_inconsistent = pd.read_parquet('C:/Users/Jorda/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_6.parquet.gzip')

In [ ]:
(sum((df_1_inconsistent['zipInconsistent'] == 1.0) & (df_1_inconsistent['oneWrong'] == 1.0)) + 
 sum((df_3_inconsistent['zipInconsistent'] == 1.0) & (df_3_inconsistent['oneWrong'] == 1.0)) +
 sum((df_5_inconsistent['zipInconsistent'] == 1.0) & (df_5_inconsistent['oneWrong'] == 1.0)))

In [ ]:
(sum((df_1_inconsistent['stateInconsistent'] == 1.0) & (df_1_inconsistent['oneWrong'] == 1.0)) + 
  sum((df_2_inconsistent['stateInconsistent'] == 1.0) & (df_2_inconsistent['oneWrong'] == 1.0)) +
 sum((df_3_inconsistent['stateInconsistent'] == 1.0) & (df_3_inconsistent['oneWrong'] == 1.0)) +
  sum((df_4_inconsistent['stateInconsistent'] == 1.0) & (df_4_inconsistent['oneWrong'] == 1.0)) +
 sum((df_5_inconsistent['stateInconsistent'] == 1.0) & (df_5_inconsistent['oneWrong'] == 1.0)))

In [ ]:
#df_2_inconsistent
df_2_inconsistent['reportedZipCode'] = df_2_inconsistent['reportedZipCode'].fillna(0)

#df_3_inconsistent
df_3_inconsistent['countyCode'] = df_3_inconsistent['countyCode'].fillna(0)

#df_4_inconsistent
df_4_inconsistent['reportedZipCode'] = df_4_inconsistent['reportedZipCode'].fillna(0)
df_4_inconsistent['countyCode'] = df_4_inconsistent['countyCode'].fillna(0)

#df_5_inconsistent
df_5_inconsistent['censusTract'] = df_5_inconsistent['censusTract'].fillna(0)
df_5_inconsistent['censusBlockGroupFips'] = df_5_inconsistent['censusBlockGroupFips'].fillna(0)

In [ ]:
#convert the units to string to use mapping and fix the lenght of each units
#df_2_inconsistent
df_2_inconsistent['reportedZipCode'] = df_2_inconsistent['reportedZipCode'].astype(int).astype(str)
df_2_inconsistent['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_2_inconsistent['reportedZipCode']]

#df_3_inconsistent
df_3_inconsistent['countyCode'] = [str(int(float(i))) for i in df_3_inconsistent['countyCode']]
df_3_inconsistent['countyCode'] = [censusBG.zfill(5) for censusBG in df_3_inconsistent['countyCode']]

#df_4_inconsistent
df_4_inconsistent['reportedZipCode'] = df_4_inconsistent['reportedZipCode'].astype(int).astype(str)
df_4_inconsistent['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_4_inconsistent['reportedZipCode']]

df_4_inconsistent['countyCode'] = [str(int(float(i))) for i in df_4_inconsistent['countyCode']]
df_4_inconsistent['countyCode'] = [censusBG.zfill(5) for censusBG in df_4_inconsistent['countyCode']]

#df_5_inconsistent
df_5_inconsistent['censusBlockGroupFips'] = [str(int(float(i))) for i in df_5_inconsistent['censusBlockGroupFips']]
df_5_inconsistent['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_5_inconsistent['censusBlockGroupFips']]

df_5_inconsistent['censusTract'] = [str(int(float(i))) for i in df_5_inconsistent['censusTract']]
df_5_inconsistent['censusTract'] = [censusBG.zfill(11) for censusBG in df_5_inconsistent['censusTract']]

In [ ]:
df_1_inconsistent.drop(columns=['zip_geometry','lat_long_geometry','bg_geometry'], inplace=True)
df_2_inconsistent.drop(columns=['lat_long_geometry','state_geometry','county_geometry'], inplace=True)
df_3_inconsistent.drop(columns=['state_geometry','zip_geometry'], inplace=True)

In [ ]:
df_geographic_unique.head()

In [ ]:
df_1_inconsistent.head()

In [ ]:
print(df_1_inconsistent.index.is_unique)


In [ ]:
# Columns that were intended to be set as index
index_columns = ['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                 'latitude', 'longitude', 'year']

# Ensure the DataFrame index is reset to default
df_reset = df_1_inconsistent.reset_index()

# Find duplicate rows based on the intended index columns
duplicates_df = df_reset[df_reset.duplicated(subset=index_columns, keep=False)]

# If you want to display only the first few instances of duplicates, you can use `.head()`
print(duplicates_df.head())  # Adjust the number inside head() to control the number of rows displayed

# If there are many duplicates and you want to see all, you would simply print the variable:
# print(duplicates_df)


In [ ]:
df_1_inconsistent.drop_duplicates(subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude','year', 'year_zipcode'], inplace=True)

# Initial mapping with multi-index
df_1_inconsistent.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year', 'year_zipcode'], inplace=True)

df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 
                                'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin'], inplace=True)

df_geographic_unique['zipInconsistent'] = df_geographic_unique.index.map(df_1_inconsistent['zipInconsistent'])

# Reset the index
df_geographic_unique.reset_index(inplace=True)


df_geographic_unique['zipInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['zipInconsistent'])
df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['cbgInconsistent'])
df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['tractInconsistent'])
df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['stateInconsistent'])
df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['countyInconsistent'])
df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['latlongInconsistent'])
df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['multiple'])
df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['oneWrong'])

# Resetting the index (return to multi-index)
df_1_inconsistent.reset_index(inplace=True)

In [ ]:
df_2_inconsistent.drop_duplicates(subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude','year'], inplace=True)

In [ ]:
# Initial mapping with multi-index
df_2_inconsistent.set_index(['reportedZipCode','censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year'], inplace=True)

multi_index = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index
cbgInconsistent = pd.Series(multi_index.map(df_2_inconsistent['cbgInconsistent']), index=multi_index)
tractInconsistent = pd.Series(multi_index.map(df_2_inconsistent['tractInconsistent']), index=multi_index)
stateInconsistent = pd.Series(multi_index.map(df_2_inconsistent['stateInconsistent']), index=multi_index)
countyInconsistent = pd.Series(multi_index.map(df_2_inconsistent['countyInconsistent']), index=multi_index)
latlongInconsistent = pd.Series(multi_index.map(df_2_inconsistent['latlongInconsistent']), index=multi_index)
multiple_inconsistent = pd.Series(multi_index.map(df_2_inconsistent['multiple']), index=multi_index)
single_inconsistent = pd.Series(multi_index.map(df_2_inconsistent['oneWrong']), index=multi_index)

# Revert df_2_inconsistent back to its original form
df_2_inconsistent.reset_index(inplace=True)

# Fill NaN values in df_geographic_unique['cbgInconsistent'] with values from cbgInconsistent Series
df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 'latitude', 
                                'longitude', 'yearOfLoss_1990_2021'], inplace=True)
df_geographic_unique['cbgInconsistent'].fillna(cbgInconsistent, inplace=True)
df_geographic_unique['tractInconsistent'].fillna(tractInconsistent, inplace=True)
df_geographic_unique['stateInconsistent'].fillna(stateInconsistent, inplace=True)
df_geographic_unique['countyInconsistent'].fillna(countyInconsistent, inplace=True)
df_geographic_unique['latlongInconsistent'].fillna(latlongInconsistent, inplace=True)
df_geographic_unique['multiple_inconsistent'].fillna(multiple_inconsistent, inplace=True)
df_geographic_unique['single_inconsistent'].fillna(single_inconsistent, inplace=True)

# Reset index for df_geographic_unique if needed
df_geographic_unique.reset_index(inplace=True)

In [ ]:
# # Initial mapping with multi-index
# df_2_inconsistent.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude'], inplace=True)

# df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['cbgInconsistent'])
# df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['tractInconsistent'])
# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['stateInconsistent'])
# df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['countyInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['latlongInconsistent'])
# df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['multiple'])
# df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['oneWrong'])

# # Resetting the index of County_df (return to multi-index)
# df_2_inconsistent.reset_index(inplace=True)

In [ ]:
df_3_inconsistent.drop_duplicates(subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude','year', 'year_zipcode'], inplace=True)

# Initial mapping with multi-index
df_3_inconsistent.set_index(['reportedZipCode','censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year', 'year_zipcode'], inplace=True)

multi_index = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index
zipInconsistent = pd.Series(multi_index.map(df_3_inconsistent['zipInconsistent']), index=multi_index)
cbgInconsistent = pd.Series(multi_index.map(df_3_inconsistent['cbgInconsistent']), index=multi_index)
tractInconsistent = pd.Series(multi_index.map(df_3_inconsistent['tractInconsistent']), index=multi_index)
stateInconsistent = pd.Series(multi_index.map(df_3_inconsistent['stateInconsistent']), index=multi_index)
# countyInconsistent = pd.Series(multi_index.map(df_3_inconsistent['countyInconsistent']), index=multi_index)
latlongInconsistent = pd.Series(multi_index.map(df_3_inconsistent['latlongInconsistent']), index=multi_index)
multiple_inconsistent = pd.Series(multi_index.map(df_3_inconsistent['multiple']), index=multi_index)
single_inconsistent = pd.Series(multi_index.map(df_3_inconsistent['oneWrong']), index=multi_index)

# Revert df_3_inconsistent back to its original form
df_3_inconsistent.reset_index(inplace=True)

# Fill NaN values in df_geographic_unique['cbgInconsistent'] with values from cbgInconsistent Series
df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 'latitude', 
                                'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin'], inplace=True)
df_geographic_unique['zipInconsistent'].fillna(zipInconsistent, inplace=True)
df_geographic_unique['cbgInconsistent'].fillna(cbgInconsistent, inplace=True)
df_geographic_unique['tractInconsistent'].fillna(tractInconsistent, inplace=True)
df_geographic_unique['stateInconsistent'].fillna(stateInconsistent, inplace=True)
# df_geographic_unique['countyInconsistent'].fillna(countyInconsistent, inplace=True)
df_geographic_unique['latlongInconsistent'].fillna(latlongInconsistent, inplace=True)
df_geographic_unique['multiple_inconsistent'].fillna(multiple_inconsistent, inplace=True)
df_geographic_unique['single_inconsistent'].fillna(single_inconsistent, inplace=True)

# Reset index for df_geographic_unique if needed
df_geographic_unique.reset_index(inplace=True)

In [ ]:
# # Initial mapping with multi-index
# df_3_inconsistent.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'year'], inplace=True)

# df_geographic_unique['zipInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['zipInconsistent'])
# df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['cbgInconsistent'])
# df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['tractInconsistent'])
# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['stateInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['latlongInconsistent'])
# df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['multiple'])
# df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['oneWrong'])

# # Resetting the index of County_df (return to multi-index)
# df_3_inconsistent.reset_index(inplace=True)

In [ ]:
df_4_inconsistent.drop_duplicates(subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude','year'], inplace=True)

In [ ]:
# Initial mapping with multi-index
df_4_inconsistent.set_index(['reportedZipCode','censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year'], inplace=True)

multi_index = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index
# zipInconsistent = pd.Series(multi_index.map(df_4_inconsistent['zipInconsistent']), index=multi_index)
cbgInconsistent = pd.Series(multi_index.map(df_4_inconsistent['cbgInconsistent']), index=multi_index)
tractInconsistent = pd.Series(multi_index.map(df_4_inconsistent['tractInconsistent']), index=multi_index)
stateInconsistent = pd.Series(multi_index.map(df_4_inconsistent['stateInconsistent']), index=multi_index)
# countyInconsistent = pd.Series(multi_index.map(df_3_inconsistent['countyInconsistent']), index=multi_index)
latlongInconsistent = pd.Series(multi_index.map(df_4_inconsistent['latlongInconsistent']), index=multi_index)
multiple_inconsistent = pd.Series(multi_index.map(df_4_inconsistent['multiple']), index=multi_index)
single_inconsistent = pd.Series(multi_index.map(df_4_inconsistent['oneWrong']), index=multi_index)

# Revert df_4_inconsistent back to its original form
df_4_inconsistent.reset_index(inplace=True)

# Fill NaN values in df_geographic_unique['cbgInconsistent'] with values from cbgInconsistent Series
df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 'latitude', 
                                'longitude', 'yearOfLoss_1990_2021'], inplace=True)
# df_geographic_unique['zipInconsistent'].fillna(zipInconsistent, inplace=True)
df_geographic_unique['cbgInconsistent'].fillna(cbgInconsistent, inplace=True)
df_geographic_unique['tractInconsistent'].fillna(tractInconsistent, inplace=True)
df_geographic_unique['stateInconsistent'].fillna(stateInconsistent, inplace=True)
# df_geographic_unique['countyInconsistent'].fillna(countyInconsistent, inplace=True)
df_geographic_unique['latlongInconsistent'].fillna(latlongInconsistent, inplace=True)
df_geographic_unique['multiple_inconsistent'].fillna(multiple_inconsistent, inplace=True)
df_geographic_unique['single_inconsistent'].fillna(single_inconsistent, inplace=True)

# Reset index for df_geographic_unique if needed
df_geographic_unique.reset_index(inplace=True)

In [ ]:
# # Initial mapping with multi-index
# df_4_inconsistent.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude'], inplace=True)

# df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['cbgInconsistent'])
# df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['tractInconsistent'])
# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['stateInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['latlongInconsistent'])
# df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['multiple'])
# df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['oneWrong'])

# # Resetting the index of County_df (return to multi-index)
# df_4_inconsistent.reset_index(inplace=True)

In [ ]:
df_5_inconsistent.drop_duplicates(subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude','year','year_zipcode'], inplace=True)

# Initial mapping with multi-index
df_5_inconsistent.set_index(['reportedZipCode','censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year', 'year_zipcode'], inplace=True)

multi_index = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index
zipInconsistent = pd.Series(multi_index.map(df_5_inconsistent['zipInconsistent']), index=multi_index)
# cbgInconsistent = pd.Series(multi_index.map(df_5_inconsistent['cbgInconsistent']), index=multi_index)
# tractInconsistent = pd.Series(multi_index.map(df_5_inconsistent['tractInconsistent']), index=multi_index)
stateInconsistent = pd.Series(multi_index.map(df_5_inconsistent['stateInconsistent']), index=multi_index)
countyInconsistent = pd.Series(multi_index.map(df_5_inconsistent['countyInconsistent']), index=multi_index)
latlongInconsistent = pd.Series(multi_index.map(df_5_inconsistent['latlongInconsistent']), index=multi_index)
multiple_inconsistent = pd.Series(multi_index.map(df_5_inconsistent['multiple']), index=multi_index)
single_inconsistent = pd.Series(multi_index.map(df_5_inconsistent['oneWrong']), index=multi_index)

# Revert df_5_inconsistent back to its original form
df_5_inconsistent.reset_index(inplace=True)

# Fill NaN values in df_geographic_unique['cbgInconsistent'] with values from cbgInconsistent Series
df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 'latitude', 
                                'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin'], inplace=True)
df_geographic_unique['zipInconsistent'].fillna(zipInconsistent, inplace=True)
# df_geographic_unique['cbgInconsistent'].fillna(cbgInconsistent, inplace=True)
# df_geographic_unique['tractInconsistent'].fillna(tractInconsistent, inplace=True)
df_geographic_unique['stateInconsistent'].fillna(stateInconsistent, inplace=True)
df_geographic_unique['countyInconsistent'].fillna(countyInconsistent, inplace=True)
df_geographic_unique['latlongInconsistent'].fillna(latlongInconsistent, inplace=True)
df_geographic_unique['multiple_inconsistent'].fillna(multiple_inconsistent, inplace=True)
df_geographic_unique['single_inconsistent'].fillna(single_inconsistent, inplace=True)

# Reset index for df_geographic_unique if needed
df_geographic_unique.reset_index(inplace=True)

In [ ]:
# # Initial mapping with multi-index
# df_5_inconsistent.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'year'], inplace=True)

# df_geographic_unique['zipInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['zipInconsistent'])
# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['stateInconsistent'])
# df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['countyInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['latlongInconsistent'])
# df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['multiple'])
# df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['oneWrong'])

# # Resetting the index of County_df (return to multi-index)
# df_5_inconsistent.reset_index(inplace=True)

In [ ]:
# # Initial mapping with multi-index (df_6_inconsistent is empty)
# df_6_inconsistent.set_index(['state', 'countyCode',
#                             'latitude', 'longitude'], inplace=True)

# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['state', 'countyCode',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['stateInconsistent'])
# df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['state', 'countyCode',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['countyInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['state', 'countyCode',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['latlongInconsistent'])

# # Resetting the index of County_df (return to multi-index)
# df_6_inconsistent.reset_index(inplace=True)

In [ ]:
df_geographic_unique[df_geographic_unique['zipInconsistent'].notna()].head()

In [ ]:
test = df_geographic_unique[df_geographic_unique['single_inconsistent'] == 1.0]

print(f"zipInconsistent: {sum(test['zipInconsistent'] == 1.0)}")
print(f"cbgInconsistent: {sum(test['cbgInconsistent'] == 1.0)}")
print(f"tractInconsistent: {sum((test['tractInconsistent'] == 1.0))}")
print(f"stateInconsistent: {sum((test['stateInconsistent'] == 1.0))}")
print(f"countyInconsistent: {sum((test['countyInconsistent'] == 1.0))}")
print(f"latlongInconsistent: {sum((test['latlongInconsistent'] == 1.0))}")
print(f"multiple_inconsistent: {sum((test['multiple_inconsistent'] == 1.0))}")
print(f"single_inconsistent: {sum((test['single_inconsistent'] == 1.0))}")

In [ ]:
print(f"zipInconsistent: {sum((df_geographic_unique['zipInconsistent'].notna()) & (df_geographic_unique['zipInconsistent'] == 1.0))}")
print(f"cbgInconsistent: {sum((df_geographic_unique['cbgInconsistent'].notna()) & (df_geographic_unique['cbgInconsistent'] == 1.0))}")
print(f"tractInconsistent: {sum((df_geographic_unique['tractInconsistent'].notna()) & (df_geographic_unique['tractInconsistent'] == 1.0))}")
print(f"stateInconsistent: {sum((df_geographic_unique['stateInconsistent'].notna()) & (df_geographic_unique['stateInconsistent'] == 1.0))}")
print(f"countyInconsistent: {sum((df_geographic_unique['countyInconsistent'].notna()) & (df_geographic_unique['countyInconsistent'] == 1.0))}")
print(f"latlongInconsistent: {sum((df_geographic_unique['latlongInconsistent'].notna()) & (df_geographic_unique['latlongInconsistent'] == 1.0))}")
print(f"multiple_inconsistent: {sum((df_geographic_unique['multiple_inconsistent'].notna()) & (df_geographic_unique['multiple_inconsistent'] == 1.0))}")
print(f"single_inconsistent: {sum((df_geographic_unique['single_inconsistent'].notna()) & (df_geographic_unique['single_inconsistent'] == 1.0))}")

In [ ]:
#TODO: for all NA values change it to 1
#TODO: create 'multipleInconsistent' column where add all flags
#TODO: for state = 'UN', make it N
#TODO (Not here but later): for missing geometries, change flag to M

In [ ]:
df_geographic_unique['zipInconsistent'].value_counts()

In [ ]:
df_geographic_unique.head()

In [ ]:
count_none =(df_geographic_unique['geometry_lat_long'] == None).sum()
print(count_none)

In [ ]:
def is_empty_or_none(geometry):
    return geometry is None or geometry.is_empty


In [ ]:
def is_empty_or_none_or_no_area(geometry):
    return (geometry is None) or (geometry.is_empty) or (geometry.area == 0)

In [ ]:
assignment_counts = {
    "bg_zip_not_empty": 0,
    "bg_no_zip_before_2000": 0,
    "no_bg_zip_after_2000": 0,
    "no_bg_no_zip": 0,
    "lat_long_none": 0,
    "lat_long_bg_no_zip_before_2000_1": 0,
    "lat_long_bg_no_zip_before_2000_2": 0,
    "lat_long_bg_zip_after_2000": 0,
    "lat_long_no_bg_zip_after_2000":0,
    "final_lat_long": 0,
    "lat_long_zip_after_2000" : 0,
    "lat_long_only_1" :0
}

In [ ]:
df_geographic_unique[df_geographic_unique['year'] < 1981]['geometry_zipcode'] = None

### Creating new geometry

In [ ]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df = gpd.GeoDataFrame(columns=['reportedZipCode', 'countyCode', 'censusTract','censusBlockGroupFips', 'latitude', 
                                        'longitude', 'state', 'new_geographic_unit', 'year', 'zip_year_bin', 'geometry_zipcode',
                                       'geometry_county', 'geometry_tract','geometry_BG','geometry_lat_long','geometry_state', 
                                        'geometry_new_unit', 'cbgInconsistent', 'tractInconsistent', 'countyInconsistent', 
                                        'stateInconsistent', 'latlongInconsistent', 'zipcodeInconsistent','multipleinconsistent','singleinconsistent'])

empty = Polygon()

# Iterate through each row in BG_df_1990 and each row in lat_long_df to find intersections
for idx_unit, row_unit in df_geographic_unique.iterrows():
    year = row_unit['yearOfLoss_1990_2021']
    year_zipcode = row_unit['zip_year_bin']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    cbgInconsistent = row_unit['cbgInconsistent']
    tractInconsistent = row_unit['tractInconsistent']
    countyInconsistent = row_unit['countyInconsistent']
    stateInconsistent = row_unit['stateInconsistent']
    latlongInconsistent = row_unit['latlongInconsistent']
    zipcodeInconsistent = row_unit['zipInconsistent']
    multipleInconsistent = row_unit['multiple_inconsistent']
    singleInconsistent = row_unit['single_inconsistent']
    new_geographic_unit = '0'
    intersection_geometry = empty

    if (singleInconsistent == 1) and (latlongInconsistent == 1):

        if (bg_geometry is not None) and (zipcode_geometry is not None):
            intersection_geometry = bg_geometry.intersection(zipcode_geometry)
            assignment_counts["bg_zip_not_empty"] += 1


            if intersection_geometry.is_empty:
                print("OH NO!! number 1")
                break
            else:
                print("bg zip not empty")
                new_geographic_unit = f"{'0'*4}{'0'*4}{bg_id:12}{zipcode:5}{year_zipcode:4}"
                
        elif (bg_geometry is not None) and ((zipcode_geometry is None)):
            new_geographic_unit = f"{'0'*4}{'0'*4}{bg_id:12}{'0'*5}{'0'*4}"
            assignment_counts["bg_no_zip_before_2000"] += 1
            intersection_geometry = bg_geometry
                
        elif (bg_geometry is None) and (zipcode_geometry is not None):
            new_geographic_unit = f"{'0'*4}{'0'*4}{'0'*12}{zipcode:5}{year_zipcode:4}"
            assignment_counts["no_bg_zip_after_2000"] += 1
            intersection_geometry = zipcode_geometry

        
        else:
            new_geographic_unit = f"{'0'*4}{'0'*4}{'0'*12}{'0'*5}{'0'*4}"
            assignment_counts["no_bg_no_zip"] += 1
            intersection_geometry = empty
                
            
            
    else:     
  
        # Check for the lat_long_geometry being None
        if lat_long_geometry is None:
            new_geographic_unit = f"{'0'*4}{'0'*4}{'0'*12}{'0'*5}{'0'*4}"
            assignment_counts["lat_long_none"] += 1
            intersection_geometry = empty
    
        elif (bg_geometry is None and zipcode_geometry is None):
            new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{'0'*5}{'0'*4}"
            assignment_counts["lat_long_only_1"] += 1
            intersection_geometry = lat_long_geometry

        elif ((zipcode_geometry is None ) and (not is_empty_or_none(lat_long_geometry.intersection(bg_geometry)))):
            new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{bg_id:12}{'0'*5}{'2010':4}"
            assignment_counts["lat_long_bg_no_zip_before_2000_1"] += 1
            intersection_geometry = lat_long_geometry.intersection(bg_geometry)

        elif (bg_geometry is None and (zipcode_geometry is not None) and (not is_empty_or_none(lat_long_geometry.intersection(zipcode_geometry)))):
            new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{zipcode:5}{year_zipcode:4}"
            assignment_counts["lat_long_zip_after_2000"] += 1
            intersection_geometry = lat_long_geometry.intersection(zipcode_geometry)

    
        elif ((bg_geometry is not None) and (zipcode_geometry is not None)):
            if not is_empty_or_none(lat_long_geometry.intersection(bg_geometry).intersection(zipcode_geometry)):
                new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{bg_id:12}{zipcode:5}{year_zipcode:4}"
                assignment_counts["lat_long_bg_zip_after_2000"] += 1
                intersection_geometry = lat_long_geometry.intersection(bg_geometry).intersection(zipcode_geometry)
        
            elif not is_empty_or_none(lat_long_geometry.intersection(bg_geometry)):
                new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{'0'*5}{'0'*4}"
                assignment_counts["lat_long_bg_no_zip_before_2000_2"] +=1
                intersection_geometry = lat_long_geometry.intersection(bg_geometry)
        
            elif not is_empty_or_none(lat_long_geometry.intersection(zipcode_geometry)):
                new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{zipcode:5}{year_zipcode:4}"
                assignment_counts['lat_long_no_bg_zip_after_2000'] +=1
                intersection_geometry = lat_long_geometry.intersection(zipcode_geometry)
        else:
                new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{'0'*5}{'0'*4}"
                assignment_counts['final_lat_long'] +=1
                intersection_geometry = lat_long_geometry

    
    new_unit_df = pd.concat([new_unit_df, pd.DataFrame({
        'reportedZipCode': [zipcode],
        'countyCode': [county_id],
        'censusTract': [tract_id],
        'censusBlockGroupFips': [bg_id],
        'latitude': [lat],
        'longitude': [long],
        'state': [state],
        'new_geographic_unit': [new_geographic_unit],
        'year': [year],
        'year_zipcode': [year_zipcode],
        'geometry_zipcode': [zipcode_geometry],
        'geometry_county': [county_geometry],
        'geometry_tract': [tract_geometry],
        'geometry_BG': [bg_geometry],
        'geometry_lat_long': [lat_long_geometry],
        'geometry_state': [state_geometry],
        'geometry_new_unit': [intersection_geometry],
        'cbgInconsistent': [cbgInconsistent],
        'tractInconsistent': [tractInconsistent],
        'countyInconsistent': [countyInconsistent],
        'stateInconsistent': [stateInconsistent],
        'latlongInconsistent': [latlongInconsistent],
        'zipcodeInconsistent': [zipcodeInconsistent],
        'multipleInconsistent': [multipleInconsistent],
        'singleInconsistent': [singleInconsistent]
        })], ignore_index=True)

In [ ]:
new_unit_df.columns

In [ ]:
new_unit_df1 = new_unit_df[['geometry_new_unit','new_geographic_unit']]

new_unit_df1 = new_unit_df1.drop_duplicates()

chunk_size = 40000  # adjust based on your system's capabilities
chunks = [x for x in range(0, len(new_unit_df1), chunk_size)]

for start in chunks:
    end = start + chunk_size
    temp_df = new_unit_df1.iloc[start:end].copy()
    temp_df['geometry_new_unit'] = temp_df['geometry_new_unit'].apply(lambda geom: geom.wkt)
    temp_df.to_parquet(f"new_geographic_unit_geometry_{start}_{end}.parquet.gzip", compression='gzip')

In [ ]:
# new_unit_df1 = new_unit_df[['geometry_new_unit','new_geographic_unit']]

new_unit_df1 = new_unit_df.drop_duplicates()

chunk_size = 10000  # adjust based on your system's capabilities
chunks = [x for x in range(0, len(new_unit_df1), chunk_size)]

for start in chunks:
    end = start + chunk_size
    temp_df = new_unit_df1.iloc[start:end].copy()
    temp_df['geometry_new_unit'] = temp_df['geometry_new_unit'].apply(lambda geom: geom.wkt)
    temp_df['geometry_zipcode'] = temp_df['geometry_zipcode'].apply(lambda geom: geom.wkt)
    temp_df['geometry_county'] = temp_df['geometry_county'].apply(lambda geom: geom.wkt)
    temp_df['geometry_tract'] = temp_df['geometry_tract'].apply(lambda geom: geom.wkt)
    temp_df['geometry_BG'] = temp_df['geometry_BG'].apply(lambda geom: geom.wkt)
    temp_df['geometry_lat_long'] = temp_df['geometry_lat_long'].apply(lambda geom: geom.wkt)
    temp_df['geometry_state'] = temp_df['geometry_state'].apply(lambda geom: geom.wkt)
    temp_df.to_parquet(f"C:/Users/jorda/Box/Flood Damage PredictionProject/new_geographic_unit_geometry_all_{start}_{end}.parquet.gzip", compression='gzip')

## Visualization

In [ ]:
print(assignment_counts)

In [ ]:
len(new_unit_df)

In [ ]:
total = sum(assignment_counts.values())
print(total)

In [ ]:
new_unit_df.columns

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new_unit_df, geometry='geometry_new_unit')

# Set the current CRS to WGS 84
gdf.crs = "EPSG:4326"

# Filter for only the continental US using bounding box constraints
continental_US = gdf.cx[-125:-66, 24:49]

# Project the filtered GeoDataFrame to Web Mercator (EPSG:3857)
continental_US_projected = continental_US.to_crs(epsg=3857)

# Compute the average area in square meters
average_area_m2 = continental_US_projected.geometry.area.mean()
median_area_m2 = continental_US_projected.geometry.area.median()

# Convert the areas from square meters to square kilometers
average_area_km2 = average_area_m2 / 1e6
median_area_km2 = median_area_m2 / 1e6

print(f"Average Area: {average_area_km2} square kilometers")
print(f"Median Area: {median_area_km2} square kilometers")

# Plot the geometries using the 'Reds' colormap
continental_US_projected.plot(edgecolor="black", cmap='Reds', figsize=(10, 10))

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Continental US: Using Latitude-Longitude geometries')

plt.show()



In [ ]:

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new_unit_df, geometry='geometry_new_unit')

# Set the CRS to WGS 84 if it's not already set
gdf.crs = gdf.crs if gdf.crs else "EPSG:4326"

# Project the GeoDataFrame to a metric CRS, e.g., Web Mercator (EPSG:3857)
gdf_projected = gdf.to_crs(epsg=3857)

# Calculate the areas in square meters
gdf_projected['area_m2'] = gdf_projected['geometry_new_unit'].area

# Sort the GeoDataFrame by the area, ascending
gdf_sorted = gdf_projected.sort_values(by='area_m2', ascending=True)

# Select the top 200 smallest areas
top_200_smallest = gdf_sorted.head(100)

# Convert the areas to square kilometers and print them
areas_km2 = top_200_smallest['area_m2'] / 1e6

for idx, area in enumerate(areas_km2, 1):
    print(f"Rank {idx}: {area:.70f} km^2")


In [ ]:
gdf.columns

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new_unit_df, geometry='geometry_new_unit')

# Set the CRS to WGS 84 if it's not already set
gdf.crs = gdf.crs if gdf.crs else "EPSG:4326"

# Filter rows where the area of 'geometry_new_unit' is 0 and 'geometry_lat_long' is not empty
zero_area_with_latlong = gdf[(gdf.geometry.area == 0) & gdf['geometry_lat_long'].notnull()]

# If there are any units with area 0, plot the combined geometries for the first unit
if not zero_area_with_latlong.empty:
    unit = zero_area_with_latlong.iloc[15]
    
    # Extracting the geometries for the first unit
    geom_lat_long = unit['geometry_lat_long']
    geom_BG = unit['geometry_BG']
    geom_zipcode = unit['geometry_zipcode']
    
    # Combine the three geometries into one GeoSeries for plotting
    combined_geometries = gpd.GeoSeries([geom_lat_long, geom_BG, geom_zipcode])
    
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    combined_geometries.plot(ax=ax, color=[(1,0,0,0.05), (0,0,1,.1), (0,1,0,1)])
    print(unit['new_geographic_unit'])

    ax.set_title('Combined Geometries for Single Unit with Zero Area in geometry_new_unit')
    plt.show()
else:
    print("No units with area of 0 in 'geometry_new_unit' with non-empty 'geometry_lat_long'.")
    
# Check if there's an intersection between geom_lat_long and geom_zipcode
intersection = geom_lat_long.intersection(geom_zipcode)

if intersection.is_empty:
    print("The intersection between lat/long and zipcode geometries is empty.")
else:
    print("There's an intersection between lat/long and zipcode geometries.")



In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new_unit_df, geometry='geometry_new_unit')

# Set the CRS to WGS 84 if it's not already set
gdf.crs = gdf.crs if gdf.crs else "EPSG:4326"

# Filter rows where the area of 'geometry_new_unit' is 0 and 'geometry_lat_long' is not empty
zero_area_with_latlong = gdf[(gdf.geometry.area == 0) & gdf['geometry_lat_long'].notnull()]

# If there are any units with area 0, plot the combined geometries for the first unit
if not zero_area_with_latlong.empty:
    unit = zero_area_with_latlong.iloc[6]
    
    # Extracting the geometries for the first unit
    geom_lat_long = unit['geometry_lat_long']
    geom_BG = unit['geometry_BG']
    geom_zipcode = unit['geometry_zipcode']
    
    # Combine the three geometries into one GeoSeries for plotting
    combined_geometries = gpd.GeoSeries([geom_zipcode])
    
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    combined_geometries.plot(ax=ax, color=[(0,1,0,1)])
    
    ax.set_title('Combined Geometries for Single Unit with Zero Area in geometry_new_unit')
    
    plt.show()
else:
    print("No units with area of 0 in 'geometry_new_unit' with non-empty 'geometry_lat_long'.")


In [ ]:
gdf.columns

In [ ]:
gdf['yearOfLoss_1990_2021'].value_counts()

In [ ]:
(gdf['geometry_lat_long'] == None).sum()

In [ ]:
gdf_projected.columns

In [ ]:
# Filter rows where the area of 'geometry_new_unit' is 0 and 'geometry_lat_long' is not empty
zero_area_with_latlong = gdf[(gdf_projected['area_m2'] == 0) & (gdf_projected['geometry_lat_long']!= None)]

# Print the 'year' column values for these ro
zero_area_with_latlong['yearOfLoss_1990_2021'].value_counts()

In [ ]:
# Count the number of rows with an area of 0
count_zero_area = (gdf_projected['area_m2'] == 0 & (gdf_projected['geometry_lat_long']!= None)).sum()

print(f"Number of observations with area of 0: {count_zero_area}")

In [ ]:
6748/585007

In [ ]:
print(unit)

In [ ]:
# Function to compute mean and median areas for a geometry column
def compute_area_statistics(df, geometry_column):
    # Convert the specific column to a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=geometry_column)
    
    # Set the current CRS to WGS 84
    gdf.crs = "EPSG:4326"

    # Project to a metric-based CRS (Web Mercator in this case, but this might introduce some area distortion)
    gdf_projected = gdf.to_crs(epsg=3857)

    # Filter out missing/empty geometries
    gdf_projected = gdf_projected[gdf_projected[geometry_column].notna() & ~gdf_projected[geometry_column].is_empty]
    
    # Compute areas in square meters 
    gdf_projected['area'] = gdf_projected[geometry_column].area
    
    # Return mean and median
    return gdf_projected['area'].mean() / 1e6, gdf_projected['area'].median() / 1e6  # Convert to km^2

# Compute for each column
for column in ['geometry_lat_long', 'geometry_BG']:
    mean_area, median_area = compute_area_statistics(df_geographic_unique, column)
    print(f"Mean area for {column}: {mean_area} km^2")
    print(f"Median area for {column}: {median_area} km^2")
    print("-------------------------------")


In [ ]:
# Filter out invalid geometries
gdf_zipcode = gdf_zipcode[gdf_zipcode['geometry_zipcode'].notna()]

# Project each geometry one-by-one to avoid memory error
def safe_transform(geom):
    try:
        return geom.to_crs(epsg=3857)
    except:
        return None

gdf_zipcode['geometry_projected'] = gdf_zipcode['geometry_zipcode'].apply(safe_transform)

# Now compute areas for the projected geometries
gdf_zipcode['area_m2'] = gdf_zipcode['geometry_projected'].area

# Count the number of rows with an area of 0
count_zero_area = (gdf_zipcode['area_m2'] == 0).sum()

print(f"Number of observations in 'geometry_zipcode' with area of 0: {count_zero_area}")



In [ ]:
sum((new_unit_df['new_geographic_unit'].isnull()) | (new_unit_df['new_geographic_unit'] == ''))


In [ ]:
total_counts = new_unit_df['new_geographic_unit'].value_counts().sum()
print(total_counts)

In [ ]:
len(new_unit_df)

In [ ]:
df_geographic_unique['geometry_lat_long'].isnull().sum() + 297957


In [ ]:
# Filter out the ZIPcodes that have shape files for both 2020 and 2010
unique_zipcodes_2020 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2020]['reportedZipCode'])
unique_zipcodes_2010 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2010]['reportedZipCode'])
common_zipcodes = unique_zipcodes_2020.intersection(unique_zipcodes_2010)

# Initialize a DataFrame to store overlap information
overlap_df = pd.DataFrame(columns=['ZIPcode', 'Overlap_Area', 'Original_Area_2010', 'Area_Difference_Change'])

# Convert df_geographic_unique into a GeoDataFrame for geometry operations
gdf = gpd.GeoDataFrame(df_geographic_unique, geometry='geometry_zipcode')

for zipcode in common_zipcodes:
    geometry_2020 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2020)].iloc[0]['geometry_zipcode']
    geometry_2010 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2010)].iloc[0]['geometry_zipcode']

    # Check if either geometry is None before proceeding
    if geometry_2020 is None or geometry_2010 is None:
        continue

    original_area_2010 = geometry_2010.area
    overlap_area = geometry_2010.intersection(geometry_2020).area
    area_diff_change = overlap_area / original_area_2010 if original_area_2010 != 0 else 0  # Handle cases where original area is 0
    
    overlap_df.loc[len(overlap_df)] = {'ZIPcode': zipcode, 
                                       'Overlap_Area': overlap_area, 
                                       'Original_Area_2010': original_area_2010, 
                                       'Area_Difference_Change': area_diff_change}
print(overlap_df["Area_Difference_Change"].mean())



In [ ]:
# Filter out the ZIPcodes that have shape files for both 2000 and 2010
unique_zipcodes_2000 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2000]['reportedZipCode'])
unique_zipcodes_2010 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2010]['reportedZipCode'])
common_zipcodes = unique_zipcodes_2000.intersection(unique_zipcodes_2010)

# Initialize a DataFrame to store overlap information
overlap_df = pd.DataFrame(columns=['ZIPcode', 'Overlap_Area', 'Original_Area_2000', 'Area_Difference_Change'])

# Convert df_geographic_unique into a GeoDataFrame for geometry operations
gdf = gpd.GeoDataFrame(df_geographic_unique, geometry='geometry_zipcode')

for zipcode in common_zipcodes:
    geometry_2000 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2000)].iloc[0]['geometry_zipcode']
    geometry_2010 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2010)].iloc[0]['geometry_zipcode']

    # Check if either geometry is None before proceeding
    if geometry_2000 is None or geometry_2010 is None:
        continue

    original_area_2000 = geometry_2000.area
    overlap_area = geometry_2000.intersection(geometry_2010).area
    area_diff_change = overlap_area / original_area_2000 if original_area_2000 != 0 else 0  # Handle cases where original area is 0
    
    overlap_df.loc[len(overlap_df)] = {'ZIPcode': zipcode, 
                                       'Overlap_Area': overlap_area, 
                                       'Original_Area_2000': original_area_2000, 
                                       'Area_Difference_Change': area_diff_change}
print(overlap_df["Area_Difference_Change"].mean())


In [ ]:
# Filter out the ZIPcodes that have shape files for both 2000 and 2010
unique_zipcodes_2000 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2000]['reportedZipCode'])
unique_zipcodes_2010 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2010]['reportedZipCode'])
common_zipcodes = unique_zipcodes_2000.intersection(unique_zipcodes_2010)

# Initialize a DataFrame to store overlap information
overlap_df = pd.DataFrame(columns=['ZIPcode', 'Overlap_Area', 'Original_Area_2000', 'Area_Difference_Change'])

# Convert df_geographic_unique into a GeoDataFrame for geometry operations
gdf = gpd.GeoDataFrame(df_geographic_unique, geometry='geometry_zipcode')

tolerance = 0.001  # This is an arbitrary value and might need adjustingfor zipcode in common_zipcodes:
for zipcode in common_zipcodes:

    geometry_2000 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2000)].iloc[0]['geometry_zipcode']
    geometry_2010 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2010)].iloc[0]['geometry_zipcode']

    # Check if either geometry is None before proceeding
    if geometry_2000 is None or geometry_2010 is None:
        continue

    # Simplify the geometry before buffering
    simplified_geometry_2000 = geometry_2000.simplify(tolerance)
    
    original_area_2000 = simplified_geometry_2000.area
    buffer_distance = (original_area_2000 * 0.10) ** 0.5  # Compute the buffer distance
    buffered_geometry_2000 = simplified_geometry_2000.buffer(buffer_distance)

    overlap_area = buffered_geometry_2000.intersection(geometry_2010).area
    final_area_2010 = geometry_2010.area
    overlap_to_final_ratio = overlap_area / final_area_2010 if final_area_2010 != 0 else 0  # Handle cases where final area is 0
    
    overlap_df.loc[len(overlap_df)] = {'ZIPcode': zipcode, 
                                       'Overlap_Area': overlap_area, 
                                       'Original_Area_2000': original_area_2000, 
                                       'Area_Difference_Change': overlap_to_final_ratio}  # Adjusted column name to reflect change

print(overlap_df["Area_Difference_Change"].mean())

In [ ]:
# Filter out the ZIPcodes that have shape files for both 2000 and 2010
unique_zipcodes_2000 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2000]['reportedZipCode'])
unique_zipcodes_2010 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2010]['reportedZipCode'])
common_zipcodes = unique_zipcodes_2000.intersection(unique_zipcodes_2010)

# Initialize a DataFrame to store overlap information
overlap_df = pd.DataFrame(columns=['ZIPcode', 'Overlap_Area', 'Original_Area_2000', 'Area_Difference_Change'])

# Convert df_geographic_unique into a GeoDataFrame for geometry operations
gdf = gpd.GeoDataFrame(df_geographic_unique, geometry='geometry_zipcode')

for zipcode in common_zipcodes:

    geometry_2000 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2000)].iloc[0]['geometry_zipcode']
    geometry_2010 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2010)].iloc[0]['geometry_zipcode']

    # Check if either geometry is None before proceeding
    if geometry_2000 is None or geometry_2010 is None:
        continue

    original_area_2000 = geometry_2000.area
    overlap_area = geometry_2000.intersection(geometry_2010).area
    final_area_2010 = geometry_2010.area
    overlap_to_final_ratio = overlap_area / final_area_2010 if final_area_2010 != 0 else 0  # Handle cases where final area is 0

    overlap_df.loc[len(overlap_df)] = {'ZIPcode': zipcode, 
                                       'Overlap_Area': overlap_area, 
                                       'Original_Area_2000': original_area_2000, 
                                       'Area_Difference_Change': overlap_to_final_ratio}

print(overlap_df["Area_Difference_Change"].mean())


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new_unit_df, geometry='geometry_new_unit')

# Set the CRS to WGS 84 if it's not already set
gdf.crs = gdf.crs if gdf.crs else "EPSG:4326"

# Filter rows where the area of 'geometry_new_unit' is 0 and 'geometry_lat_long' is not empty
zero_area_with_latlong = gdf[(gdf.geometry.area == 0) & gdf['geometry_lat_long'].notnull()]

# Calculate the number of zero area values with non-empty zip geometry
num_zero_area_with_latlong = len(zero_area_with_latlong)
print(f"Number of zero area values with non-empty 'geometry_lat_long': {num_zero_area_with_latlong}")

# If there are any units with area 0, plot the combined geometries for the first unit
if not zero_area_with_latlong.empty:
    unit = zero_area_with_latlong.iloc[0]
    
    # Extracting the geometries for the first unit
    geom_lat_long = unit['geometry_lat_long']
    geom_BG = unit['geometry_BG']
    geom_zipcode = unit['geometry_zipcode']
    
    # Combine the three geometries into one GeoSeries for plotting
    combined_geometries = gpd.GeoSeries([geom_lat_long, geom_BG, geom_zipcode])
    
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    combined_geometries.plot(ax=ax, color=[(1,0,0,0.5), (0,0,1,0.9), (0,1,0,0.5)])
    
    ax.set_title('Combined Geometries for Single Unit with Zero Area in geometry_new_unit')
    
    plt.show()
else:
    print("No units with area of 0 in 'geometry_new_unit' with non-empty 'geometry_lat_long'.")


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new_unit_df, geometry='geometry_new_unit')

# Set the CRS to WGS 84 if it's not already set
gdf.crs = gdf.crs if gdf.crs else "EPSG:4326"

# Filter rows where the area of 'geometry_new_unit' is 0 and 'geometry_zipcode' is not empty
zero_area_with_zip = gdf[(gdf.geometry.area == 0) & gdf['geometry_zipcode'].notnull()]

# Calculate the number of zero area values with non-empty zip geometry
num_zero_area_with_zip = len(zero_area_with_zip)
print(f"Number of zero area values with non-empty zip code: {num_zero_area_with_zip}")

# If there are any units with area 0, plot the combined geometries for the first unit
if not zero_area_with_zip.empty:
    unit = zero_area_with_zip.iloc[0]
    
    # Extracting the geometries for the first unit
    geom_lat_long = unit['geometry_lat_long']
    geom_BG = unit['geometry_BG']
    geom_zipcode = unit['geometry_zipcode']
    
    # Combine the three geometries into one GeoSeries for plotting
    combined_geometries = gpd.GeoSeries([geom_lat_long, geom_BG, geom_zipcode])
    
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    combined_geometries.plot(ax=ax, color=[(1,0,0,0.5), (0,0,1,0.9), (0,1,0,0.5)])
    
    ax.set_title('Combined Geometries for Single Unit with Zero Area in geometry_new_unit')
    
    plt.show()
else:
    print("No units with area of 0 in 'geometry_new_unit' with non-empty zip code.")


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new_unit_df, geometry='geometry_new_unit')

# Set the CRS to WGS 84 if it's not already set
gdf.crs = gdf.crs if gdf.crs else "EPSG:4326"

# Filter rows where the area of 'geometry_new_unit' is 0, 'geometry_lat_long' is not empty, and 'geometry_zipcode' is not empty
zero_area_with_latlong_zip = gdf[(gdf.geometry.area == 0) & gdf['geometry_lat_long'].notnull() & gdf['geometry_zipcode'].notnull()]

# Calculate the number of zero area values with non-empty lat-long and zip geometry
num_zero_area_with_latlong_zip = len(zero_area_with_latlong_zip)
print(f"Number of zero area values with non-empty lat-long and zip code: {num_zero_area_with_latlong_zip}")

# If there are any such units, plot the combined geometries for the first unit
if not zero_area_with_latlong_zip.empty:
    unit = zero_area_with_latlong_zip.iloc[0]
    
    # Extracting the geometries for the first unit
    geom_lat_long = unit['geometry_lat_long']
    geom_BG = unit['geometry_BG']
    geom_zipcode = unit['geometry_zipcode']
    
    # Combine the three geometries into one GeoSeries for plotting
    combined_geometries = gpd.GeoSeries([geom_lat_long, geom_BG, geom_zipcode])
    
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    combined_geometries.plot(ax=ax, color=[(1,0,0,0.5), (0,0,1,0.9), (0,1,0,0.5)])
    
    ax.set_title('Combined Geometries for Single Unit with Zero Area in geometry_new_unit')
    
    plt.show()
else:
    print("No units with area of 0 in 'geometry_new_unit' with non-empty lat-long and zip code.")


In [ ]:
import geopandas as gpd

# Convert the DataFrame to a GeoDataFrame if it's not already one
if not isinstance(df_geographic_unique, gpd.GeoDataFrame):
    df_geographic_unique = gpd.GeoDataFrame(df_geographic_unique)

# Filter rows where 'geometry_lat_long' is empty but 'geometry_zipcode' is not empty
no_latlong_with_zip = df_geographic_unique[df_geographic_unique['geometry_lat_long'].isnull() & df_geographic_unique['geometry_zipcode'].notnull()]

# Calculate the number of such observations
num_no_latlong_with_zip = len(no_latlong_with_zip)
print(f"Number of observations without lat-long but with zip code: {num_no_latlong_with_zip}")


In [ ]:

new_unit_df1 = new_unit_df1.drop_duplicates()

chunk_size = 40000  # adjust based on your system's capabilities
chunks = [x for x in range(0, len(new_unit_df1), chunk_size)]

for start in chunks:
    end = start + chunk_size
    temp_df = new_unit_df1.iloc[start:end].copy()
    temp_df['geometry'] = temp_df['geometry'].apply(lambda geom: geom.wkt)
    temp_df.to_parquet(f"new_geographic_unit_geometry_{start}_{end}.parquet.gzip", compression='gzip')

In [ ]:
import gc
import geopandas as gpd
import matplotlib.pyplot as plt


true = True
if(true):
    visualize = df_geographic_unique[df_geographic_unique['state'] == 'NC']

    # Explicitly create GeoSeries from the 'geometry_BG' column
    geometry_bg = gpd.GeoSeries(visualize['geometry_BG'])
    geometry_zipcode = gpd.GeoSeries(visualize['geometry_zipcode'])

    # Ensure that the GeoSeries has the correct Coordinate Reference System (CRS)
    geometry_bg.set_crs("EPSG:4326", inplace=True)
    geometry_zipcode.set_crs("EPSG:4326", inplace=True)

    # Calculate the centroids for the filtered data
    centroids = geometry_bg.centroid

    # Apply the centroid-based filter to the DataFrame
    visualize_NC_filtered = visualize[(centroids.y < 37) & (centroids.y > 22)]

    # Create GeoDataFrames for block groups and zip codes using the filtered DataFrame
    gdf_bg = gpd.GeoDataFrame(visualize_NC_filtered, geometry=geometry_bg)
    gdf_zipcode = gpd.GeoDataFrame(visualize_NC_filtered, geometry=geometry_zipcode)

    # Calculate the intersection of the two GeoDataFrames
    gdf_intersection = gpd.overlay(gdf_bg, gdf_zipcode, how='intersection')

    # Project the intersection to a suitable CRS
    gdf_intersection_projected = gdf_intersection.to_crs(epsg=3857)

    # If plotting dataset is still too large, consider sampling
    if len(gdf_intersection_projected) > 50000:  # adjust this threshold based on your system's capability
        gdf_intersection_projected = gdf_intersection_projected.sample(50000)

    # Create a base plot
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    # Plot the boundaries of intersections
    gdf_intersection_projected.boundary.plot(ax=ax, color='green', linewidth=2, label='Overlapping Boundaries')

    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Overlap between Block Groups and Zip Codes in NC')
    ax.legend()

    plt.show()

    # Cleanup
    del visualize, geometry_bg, geometry_zipcode, centroids, visualize_NC_filtered, gdf_bg, gdf_zipcode, gdf_intersection, gdf_intersection_projected
    gc.collect()
